In [ ]:
# Distance between protein and ligand

import MDAnalysis as mda
import numpy as np

# Load your topology and trajectory files
u = mda.Universe('topology', 'trajectory')

# Define selections for the protein and ligand
protein = u.select_atoms('protein')  # Select binding site atoms
ligand = u.select_atoms('resname LIG')  # Adjust 'LIG' to ligand's residue name

# Function to calculate the center of mass (COM)
def calculate_com(selection):
    mass = selection.masses
    positions = selection.positions
    com = np.sum(positions.T * mass, axis=1) / np.sum(mass)  # Weighted average of positions
    return com

# Function to calculate the distance between two COMs
def calculate_distance(com1, com2):
    return np.linalg.norm(com1 - com2)

# Calculate the distances between the protein and ligand COMs over all frames
distances = []
for ts in u.trajectory:
    protein_com = calculate_com(protein)
    ligand_com = calculate_com(ligand)
    
    # Compute the distance between the protein and ligand COMs for this frame
    distance = calculate_distance(protein_com, ligand_com)
    distances.append(distance)

# Calculate the mean distance
mean_distance = np.mean(distances)
print(f"Mean Distance Between Protein and Ligand COM: {mean_distance:.2f} Å")


In [ ]:
# Ligand atom wise contact map

import MDAnalysis as mda
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Load the topology and trajectory files
u = mda.Universe('topology', 'trajectory')

# Select atoms for the protein and ligand
protein = u.select_atoms('protein')  # All protein residues
ligand = u.select_atoms('resname LIG')  # Adjust 'LIG' to ligand's residue name

# Define a function to calculate the distance between two sets of atoms
def calculate_distances(atom_selection1, atom_selection2):
    positions1 = atom_selection1.positions
    positions2 = atom_selection2.positions
    return np.linalg.norm(positions1[:, np.newaxis, :] - positions2[np.newaxis, :, :], axis=-1)

# Create a function to compute interaction matrix
def interaction_matrix(protein, ligand, cutoff=5.0):
    # Get protein residues
    protein_residues = protein.residues
    ligand_atoms = ligand
    
    # Initialize an interaction matrix: ligand atoms vs protein residues
    matrix = np.zeros((len(ligand_atoms), len(protein_residues)))
    
    # Loop through all frames and calculate interactions
    for ts in u.trajectory:
        # Calculate pairwise distances between ligand and protein atoms
        dists = calculate_distances(ligand, protein)
        
        # Update interaction matrix
        for i, ligand_atom in enumerate(ligand_atoms):
            for j, protein_residue in enumerate(protein_residues):
                # If the distance between ligand atom and protein atom is less than the cutoff, mark as an interaction
                if np.any(dists[i, j] < cutoff):
                    matrix[i, j] += 1
    
    return matrix, ligand_atoms, protein_residues

# Generate the interaction matrix
matrix, ligand_atoms, protein_residues = interaction_matrix(protein, ligand)

# Create a heatmap
# Create labels for the heatmap (use atom names for ligand and residue names for protein)
ligand_atom_labels = [atom.name for atom in ligand_atoms]
protein_residue_labels = [residue.resname + str(residue.resid) for residue in protein_residues]

# Plot the heatmap
plt.figure(figsize=(15, 10))
sns.heatmap(matrix, xticklabels=protein_residue_labels, yticklabels=ligand_atom_labels, cmap='Paired', cbar=True)
#plt.title('Interaction Between Ligand Atoms and Protein Residues')
plt.xlabel('Protein Residue')
plt.ylabel('Ligand Atom')
plt.xticks(rotation=90)  # Rotate the x-axis labels if necessary
plt.tight_layout()
plt.savefig('ligand_protein_interaction.png')
plt.show()


In [ ]:
# Ligand residence time
import MDAnalysis as mda
from MDAnalysis.analysis import align, rms
import numpy as np

# Load your trajectory and topology
u = mda.Universe("topology", "trajectory")

# Select the ligand and protein residues
ligand = u.select_atoms("resname LIG")  # Adjust "LIG" to your ligand residue name
protein = u.select_atoms("protein")  # binding site residues

# Align the protein to its reference position
ref = u.select_atoms("protein").atoms.positions
aligner = align.AlignTraj(u, u, select="protein").run()

def is_ligand_bound(ligand, protein, distance_threshold=3.0):
    """
    Check if the ligand is bound to the protein in a frame.
    Returns True if any ligand atom is within distance_threshold of any protein atom.
    """
    distances = np.linalg.norm(ligand.positions[:, None] - protein.positions[None, :], axis=-1)
    return np.any(distances < distance_threshold)

# Parameters
distance_threshold = 3.0  # in angstroms
bound_period = 0  # time in frames during which the ligand was bound
total_bound_time = 0  # total bound time in frames
frames = len(u.trajectory)  # total number of frames

# Tracking variables
in_binding_state = False
start_frame = 0

# Iterate over each frame
for ts in u.trajectory:
    if is_ligand_bound(ligand, protein, distance_threshold):
        if not in_binding_state:
            # Ligand just bound
            in_binding_state = True
            start_frame = ts.frame
    else:
        if in_binding_state:
            # Ligand just unbound
            bound_period = ts.frame - start_frame
            total_bound_time += bound_period
            in_binding_state = False

# Account for ligand being bound at the last frame
if in_binding_state:
    bound_period = frames - start_frame
    total_bound_time += bound_period

# Average residence time (in frames)
average_residence_time = total_bound_time / frames

print(f"Total bound time (frames): {total_bound_time}")
print(f"Average residence time (frames): {average_residence_time}")

# If needed, convert frames to time units based on the simulation time step
time_step = u.trajectory.dt  # in ps
average_residence_time_ps = average_residence_time * time_step
print(f"Average residence time (ps): {average_residence_time_ps}")


In [ ]:
# Percentage of frames with ligand-residue interaction

import MDAnalysis as mda
import numpy as np

# Load the trajectory and topology
u = mda.Universe('topology', 'trajectory')

# Define the atom selections for the ligand and the residue
ligand_atoms = u.select_atoms('resname LIG and name O')  # Replace 'LIG' with ligand residue name and select atom of interest
residue_atoms = u.select_atoms('resid 40 and name ND1')  # Replace '40' with the residue ID you're interested in along with specific atom

# Define the cutoff distance based on the type of interaction, one is interested
cutoff_distance = 3.2  # in angstroms (Å)

# Initialize a counter for frames with interactions
frames_with_interaction = 0
total_frames = 0

# Loop over the frames and calculate interactions
for ts in u.trajectory:
    total_frames += 1
    interaction_found = False
    
    for ligand_atom in ligand_atoms:
        for residue_atom in residue_atoms:
            
            dist = np.linalg.norm(ligand_atom.position - residue_atom.position)
            
            # If the distance is below the cutoff, consider it an interaction
            if dist < cutoff_distance:
                interaction_found = True
                break  # No need to check further once we found an interaction
    
    if interaction_found:
        frames_with_interaction += 1

percentage_interaction = (frames_with_interaction / total_frames) * 100

print(f"Percentage of frames with ligand-residue interaction: {percentage_interaction:.2f}%")
